Import libraries

In [1]:
import zipfile
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import glob
import matplotlib.image as mpimg
import numpy as np
import random
import imageio
from IPython.display import Image
import seaborn as sns
from torchsummary import summary

In [2]:
import torch
import random

In [3]:
!pip install -q monai
!pip install -q git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
import monai
from glob import glob
from monai.transforms import Randomizable, apply_transform
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor, RandAffine
from monai.utils import get_seed

     |████████████████████████████████| 1.1 MB 14.7 MB/s 


In [4]:
import time
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm as tqdm
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
import random

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import roc_auc_score
import albumentations

import monai
from monai.data import ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, ToTensor

In [5]:
import warnings
warnings.filterwarnings("ignore")

Mount dataset

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/fyp/train5822-bmp.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset-bmp')

In [6]:
train = pd.read_csv("/content/drive/MyDrive/fyp/train-5822.csv")
len(pd.unique(train['StudyInstanceUID']))

5822

In [7]:
with zipfile.ZipFile('/content/drive/MyDrive/fyp/test1457-bmp.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset-bmp')

In [8]:
test = pd.read_csv("/content/drive/MyDrive/fyp/test-1457.csv")
len(pd.unique(test['StudyInstanceUID']))

1457

Training Setup

In [9]:
device = torch.device('cuda')

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    
set_seed(0)

In [10]:
kernel_type = 'resnet50-3d_128_2_1e-5_20ep_aug'

image_size = 126
data_dir = '/content/dataset-bmp'
num_workers = 2
init_lr = 1e-5
out_dim = 9
freeze_epo = 0
warmup_epo = 1
cosine_epo = 19
n_epochs = freeze_epo + warmup_epo + cosine_epo

In [11]:
target_cols = ['negative_exam_for_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe', 
        'chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate' ]

In [12]:
from sklearn.model_selection import GroupKFold
np.random.seed(0)
group_kfold = GroupKFold(n_splits=5)
print(group_kfold)

train['fold'] = -1
for i, (_, val_index) in enumerate(group_kfold.split(train, groups=train.StudyInstanceUID)):
    train.loc[val_index, 'fold'] = i

train.fold.value_counts()

GroupKFold(n_splits=5)


0    286305
4    286303
3    286242
1    286241
2    286241
Name: fold, dtype: int64

In [13]:
df_study = train.drop_duplicates('StudyInstanceUID')[['StudyInstanceUID','SeriesInstanceUID','fold']+target_cols]

Image Dataset Loading

In [14]:
class RSNADataset3D(torch.utils.data.Dataset, Randomizable):
    def __init__(self, csv, mode, transform=None):

        self.csv = csv.reset_index()
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return self.csv.shape[0]
    
    def randomize(self) -> None:
        MAX_SEED = np.iinfo(np.uint32).max + 1
        self._seed = self.R.randint(MAX_SEED, dtype="uint32")    

    def __getitem__(self, index):
        self.randomize()
        row = self.csv.iloc[index]
        bmp_lst = sorted(glob(os.path.join(data_dir, row.StudyInstanceUID, row.SeriesInstanceUID, '*.bmp')))
        img_lst = [cv2.imread(bmp)[:,:,::-1] for bmp in bmp_lst] 
        img = np.stack([image.astype(np.float32) for image in img_lst], axis=2).transpose(3,0,1,2)

        if self.transform is not None:
            if isinstance(self.transform, Randomizable):
                self.transform.set_random_state(seed=self._seed)
            img = apply_transform(self.transform, img)

        if self.mode == 'test':
            return img
        else:
            return img, torch.tensor(row[target_cols]).float()

In [15]:
train_transforms = Compose([ScaleIntensity(), 
                            Resize((image_size, image_size, image_size)), 
                            RandAffine( 
                                      prob=0.5,
                                      translate_range=(5, 5, 5),
                                      rotate_range=(np.pi * 4, np.pi * 4, np.pi * 4),
                                      scale_range=(0.15, 0.15, 0.15),
                                      padding_mode='border'),
                            ToTensor()])
val_transforms = Compose([ScaleIntensity(), Resize((image_size, image_size, image_size)), ToTensor()])

In [ ]:
dataset_show = RSNADataset3D(train.head(5), 'train', transform=val_transforms)
dataset_show_aug = RSNADataset3D(train.head(5), 'train', transform=train_transforms)
from pylab import rcParams
rcParams['figure.figsize'] = 20,5
for i in range(5):
    f, axarr = plt.subplots(1,6)
    img, label = dataset_show[i]
    for j in range(6):        
        if j<=2: axarr[j].imshow(img.numpy().transpose(1,2,3,0).mean(axis=j))
        elif j==3: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[image_size//2,:,:])
        elif j==4: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[:,image_size//2,:])
        elif j==5: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[:,:,image_size//2])
        axarr[j].set_title(f"Orig {i}")
    f, axarr = plt.subplots(1,6)
    img, label = dataset_show_aug[i]    
    for j in range(6):        
        if j<=2: axarr[j].imshow(img.numpy().transpose(1,2,3,0).mean(axis=j))
        elif j==3: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[image_size//2,:,:])
        elif j==4: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[:,image_size//2,:])
        elif j==5: axarr[j].imshow(img.numpy().transpose(1,2,3,0)[:,:,image_size//2])
        axarr[j].set_title(f"Aug {i}")
    plt.show()

In [ ]:
bce = nn.BCEWithLogitsLoss(reduction=('none'), pos_weight=torch.as_tensor(2.3, dtype=torch.float)).cuda()
weights = torch.tensor([0.0736196319, 0.09202453988, 0.1042944785, 0.1042944785, 0.1877300613, 0.06257668712, 0.06257668712, 0.2346625767, 0.0782208589]).view(1,-1).cuda()
def criterion(logits, target):
    total_loss = torch.tensor(0, dtype=torch.float32).to(device)
    for i in range(logits.shape[0]):
      metric = bce(logits[i,:].view(-1), target[i,:].view(-1))
      loss = torch.sum(metric.cuda()*weights, 1)[0]
      total_loss += loss
    avg_loss = total_loss/(logits.shape[0])
    return avg_loss

In [ ]:
def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(data)       
        loss = criterion(logits, target)

        loss.backward()
        
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss

In [ ]:
def val_epoch(model, loader, is_ext=None, n_test=1, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)
            LOGITS.append(logits.detach().cpu())
            TARGETS.append(target.detach().cpu())

    val_loss = criterion(torch.cat(LOGITS).cuda(), torch.cat(TARGETS).cuda()).cpu().numpy()
    PROBS = torch.sigmoid(torch.cat(LOGITS)).numpy().squeeze()    
    LOGITS = torch.cat(LOGITS).numpy()
    TARGETS = torch.cat(TARGETS).numpy()
    
    if get_output:
        return LOGITS, PROBS, TARGETS
    else:
        acc = (PROBS.round() == TARGETS).mean() * 100.
        auc = roc_auc_score(TARGETS, LOGITS)
        return float(val_loss), acc, auc

In [ ]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [ ]:
def run(fold=0):
    df_train = df_study[(df_study['fold'] != fold)]
    df_valid = df_study[(df_study['fold'] == fold)]

    dataset_train = RSNADataset3D(df_train, 'train', transform=val_transforms)
    dataset_valid = RSNADataset3D(df_valid, 'val', transform=val_transforms)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=2, sampler=RandomSampler(dataset_train), num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=2, num_workers=num_workers)

    #model = monai.networks.nets.densenet.densenet121(spatial_dims=3, in_channels=3, out_channels=out_dim).to(device)
    model = monai.networks.nets.resnet.resnet50(spatial_dims=3, n_input_channels=3, num_classes=out_dim).to(device)

    model.load_state_dict(torch.load("/content/drive/MyDrive/resnet50-3d_128_2_1e-5_20ep_aug_best (1).pth"), strict=True)

    val_loss_best = 1000
    model_file = f'{kernel_type}_best.pth'

    optimizer = optim.Adam(model.parameters(), lr=init_lr)
        
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, cosine_epo)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

    print(len(dataset_train), len(dataset_valid))

    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)
        scheduler_warmup.step(epoch-1)

        train_loss = train_epoch(model, train_loader, optimizer)
        val_loss, acc, auc = val_epoch(model, valid_loader)
    
        content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}, auc: {(auc):.6f}'
        print(content)
        with open(f'log_{kernel_type}.txt', 'a') as appender:
            appender.write(content + '\n')             
            
        if val_loss < val_loss_best:
            print('val_loss_best ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_best, val_loss))
            torch.save(model.state_dict(), model_file)
            val_loss_best = val_loss

    torch.save(model.state_dict(), f'/content/drive/MyDrive/{kernel_type}_model.pth')

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1024 B  |    1024 B  |    1024 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |    1024 B  |    1024 B  |    1024 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    1024 B  |    1024 B  |    1024 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
run()

4657 1165
Sat Sep 17 16:09:52 2022 Epoch: 1


100%|██████████| 583/583 [09:27<00:00,  1.03it/s]


Sat Sep 17 18:00:10 2022 Epoch 1, lr: 0.0000100, train loss: 0.52183, valid loss: 0.52758, acc: 85.2837, auc: 0.593650
val_loss_best (1000.000000 --> 0.527585).  Saving model ...
Sat Sep 17 18:00:11 2022 Epoch: 2


100%|██████████| 583/583 [09:29<00:00,  1.02it/s]


Sat Sep 17 19:50:23 2022 Epoch 2, lr: 0.0001000, train loss: 0.52918, valid loss: 0.53791, acc: 81.7358, auc: 0.589113
Sat Sep 17 19:50:23 2022 Epoch: 3


...

100%|██████████| 583/583 [09:29<00:00,  1.02it/s]


Sun Sep 18 12:23:04 2022 Epoch 11, lr: 0.0000541, train loss: 0.51822, valid loss: 0.50499, acc: 84.5589, auc: 0.638708
val_loss_best (0.521133 --> 0.504987).  Saving model ...
Sun Sep 18 12:23:04 2022 Epoch: 12


100%|██████████| 583/583 [09:27<00:00,  1.03it/s]


Sun Sep 18 14:13:19 2022 Epoch 12, lr: 0.0000459, train loss: 0.51081, valid loss: 0.53754, acc: 81.5546, auc: 0.552800
Sun Sep 18 14:13:19 2022 Epoch: 13


loss: 0.47695, smth: 0.48719:  64%|██████▍   | 1492/2329 [1:04:37<36:11,  2.59s/it]

In [ ]:
path = "/content"
dir_list = os.listdir(path)
filepath = [os.path.join(path, file) for file in dir_list if (file.startswith('resnet') or file.startswith('log'))]
filepath

['/content/densenet121-3d_128_2_1e-5_20ep_aug_model.pth',
 '/content/densenet121-3d_128_2_1e-5_20ep_aug_best.pth',
 '/content/log_densenet121-3d_128_2_1e-5_20ep_aug.txt']

In [ ]:
for file in filepath:
  shutil.copy(os.path.join('/content/', file),"/content/drive/MyDrive")

Inference:

In [ ]:
shutil.copy("/content/drive/MyDrive/fyp/fyp2 models/densenet121_big_dataset/densenet121-3d_8_3ch_1e-5_20ep_aug_best_fold0.pth", "/content")

'/content/densenet121-3d_8_3ch_1e-5_20ep_aug_best_fold0.pth'

In [ ]:
model_files = ['/content/densenet121-3d_8_3ch_1e-5_20ep_aug_best_fold0.pth']

In [ ]:
shutil.copy("/content/drive/MyDrive/fyp/fyp2 models/resnet50/resnet50-3d_128_16_1e-5_20ep_aug_best.pth", "/content")

'/content/resnet50-3d_128_16_1e-5_20ep_aug_best.pth'

In [32]:
model_files = ['/content/drive/MyDrive/resnet50-3d_128_2_1e-5_20ep_aug_best (4).pth']

In [33]:
def load_model(model_file):
    #model = monai.networks.nets.densenet.densenet121(spatial_dims=3, in_channels=3, out_channels=out_dim).to(device)
    model = monai.networks.nets.resnet.resnet50(spatial_dims=3, n_input_channels=3, num_classes=out_dim).to(device)

    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_file), strict=True)
    except:  # multi GPU model_file
        state_dict = torch.load(model_file)
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
        model.load_state_dict(state_dict, strict=True)

    model.eval()    
    print()
    return model

models = [load_model(model) for model in model_files]
len(models)

1

In [ ]:
densenet = monai.networks.nets.densenet.densenet121(spatial_dims=3, in_channels=3, out_channels=9).cuda()
summary(densenet, (3, 128, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 64, 64, 64]          65,856
       BatchNorm3d-2       [-1, 64, 64, 64, 64]             128
              ReLU-3       [-1, 64, 64, 64, 64]               0
         MaxPool3d-4       [-1, 64, 32, 32, 32]               0
       BatchNorm3d-5       [-1, 64, 32, 32, 32]             128
              ReLU-6       [-1, 64, 32, 32, 32]               0
            Conv3d-7      [-1, 128, 32, 32, 32]           8,192
       BatchNorm3d-8      [-1, 128, 32, 32, 32]             256
              ReLU-9      [-1, 128, 32, 32, 32]               0
           Conv3d-10       [-1, 32, 32, 32, 32]         110,592
      _DenseLayer-11       [-1, 96, 32, 32, 32]               0
      BatchNorm3d-12       [-1, 96, 32, 32, 32]             192
             ReLU-13       [-1, 96, 32, 32, 32]               0
           Conv3d-14      [-1, 128, 32,

In [ ]:
print(densenet)

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

In [ ]:
resnet = monai.networks.nets.resnet.resnet50(spatial_dims=3, n_input_channels=3, num_classes=9).cuda()
summary(resnet, (3, 128, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 64, 128, 128, 128]          65,856
       BatchNorm3d-2    [-1, 64, 128, 128, 128]             128
              ReLU-3    [-1, 64, 128, 128, 128]               0
         MaxPool3d-4       [-1, 64, 64, 64, 64]               0
            Conv3d-5       [-1, 64, 64, 64, 64]           4,096
       BatchNorm3d-6       [-1, 64, 64, 64, 64]             128
              ReLU-7       [-1, 64, 64, 64, 64]               0
            Conv3d-8       [-1, 64, 64, 64, 64]         110,592
       BatchNorm3d-9       [-1, 64, 64, 64, 64]             128
             ReLU-10       [-1, 64, 64, 64, 64]               0
           Conv3d-11      [-1, 256, 64, 64, 64]          16,384
      BatchNorm3d-12      [-1, 256, 64, 64, 64]             512
           Conv3d-13      [-1, 256, 64, 64, 64]          16,640
      BatchNorm3d-14      [-1, 256, 64,

In [ ]:
print(resnet)

ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResNetBottleneck(
      (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv3d(64, 

In [18]:
data_dir = '/content/dataset-bmp'

In [19]:
test = test.drop_duplicates('StudyInstanceUID')[['StudyInstanceUID','SeriesInstanceUID']]

In [20]:
test.shape

(1457, 2)

In [21]:
dataset_test = RSNADataset3D(test, 'test', transform=val_transforms)
test_loader = DataLoader(dataset_test, batch_size=2, num_workers=2)

In [ ]:
def load_model(model_file):
    model = monai.networks.nets.densenet.densenet121(spatial_dims=3, in_channels=3, out_channels=out_dim).to(device)

    try:  # single GPU model_file
        model.load_state_dict(torch.load(model_file), strict=True)
    except:  # multi GPU model_file
        state_dict = torch.load(model_file)
        state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
        model.load_state_dict(state_dict, strict=True)

    model.eval()    
    print()
    return model

models = [load_model(model) for model in model_files]
len(models)

In [34]:
LOGITS = []
with torch.no_grad():
    for data in tqdm(test_loader):
        data = data.to(device)
        logits = torch.zeros((data.shape[0], out_dim)).to(device)
        for model in models:
            l = model(data)
            logits += l
        logits /= len(models)
        LOGITS.append(logits.detach().cpu())
PROBS = torch.sigmoid(torch.cat(LOGITS)).numpy().squeeze()

100%|██████████| 729/729 [11:40<00:00,  1.04it/s]


In [35]:
PROBS

array([[0.58978933, 0.7975589 , 0.3309908 , ..., 0.1479348 , 0.52586347,
        0.0140153 ],
       [0.667479  , 0.5560933 , 0.3746849 , ..., 0.08312056, 0.3793561 ,
        0.04687814],
       [0.77891433, 0.31904697, 0.33992296, ..., 0.06998663, 0.12400448,
        0.07704295],
       ...,
       [0.6888086 , 0.6008456 , 0.32659537, ..., 0.10077587, 0.28565216,
        0.02373669],
       [0.7955152 , 0.27324316, 0.3475734 , ..., 0.04979713, 0.11733203,
        0.06591278],
       [0.77650696, 0.25609955, 0.39517042, ..., 0.0379441 , 0.15097277,
        0.0663597 ]], dtype=float32)

In [36]:
df = pd.DataFrame(PROBS)
df

,0,1,2,3,4,5,6,7,8
0,0.589789,0.797559,0.330991,0.715984,0.124223,0.752039,0.147935,0.525863,0.014015
1,0.667479,0.556093,0.374685,0.585003,0.072159,0.655971,0.083121,0.379356,0.046878
2,0.778914,0.319047,0.339923,0.424603,0.095430,0.488248,0.069987,0.124004,0.077043
3,0.795343,0.256724,0.349557,0.387407,0.069099,0.460778,0.052353,0.108902,0.088891
4,0.796367,0.252615,0.341286,0.378597,0.075807,0.448011,0.058946,0.106413,0.114567
...,...,...,...,...,...,...,...,...,...
1452,0.769083,0.383679,0.336835,0.467725,0.058022,0.556355,0.044140,0.173620,0.027764
1453,0.776102,0.326717,0.350947,0.433983,0.052863,0.529941,0.042119,0.156316,0.042759
1454,0.688809,0.600846,0.326595,0.579216,0.113301,0.641457,0.100776,0.285652,0.023737
1455,0.795515,0.273243,0.347573,0.405276,0.059915,0.471260,0.049797,0.117332,0.065913


In [37]:
df.to_csv('resnet50-predict4.csv', index=False)
shutil.copy("/content/resnet50-predict4.csv","/content/drive/MyDrive")

'/content/drive/MyDrive/resnet50-predict4.csv'